In [1]:
import pandas as pd
import numpy as np

https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
data_directory = "drive/My Drive/MTP/data/"

In [4]:
train_tsv = data_directory + "train.tsv"  #remove ‘content/’ from path then use 
data = pd.read_csv(train_tsv, sep='\t')

In [5]:
corpus = data['Text'].to_numpy()

In [6]:
corpus.shape

(6936,)

In [7]:
!pip install gensim

In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)


In [9]:
!pip install --upgrade language-check

  Created wheel for language-check: filename=language_check-1.1-cp36-none-any.whl size=90190900 sha256=f1304f1b1c39cd16642636a55fabdc8c0d45b93be66a7fd89219571efa765d7f
  Stored in directory: /root/.cache/pip/wheels/d5/46/82/90a89c23eac1837364ed7217a9eed71bc9e6ad4825be93968e
Successfully built language-check


In [10]:
!pip install pycontractions

In [11]:
from bs4 import BeautifulSoup
import spacy
!pip install Unidecode
import unidecode
!pip install word2number
from word2number import w2n

     |████████████████████████████████| 245kB 7.9MB/s 
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=36c455adc5a85972265082bf301b6b4c7ec3e8e30f89492d3034d8c93a710e24
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number


In [12]:
import gensim.downloader as api# load spacy model, can be "en_core_web_sm" as well
#nlp = spacy.load('en_core_web_md')# Choose model accordingly for contractions function
#model = api.load("glove-twitter-100")
# model = api.load("glove-twitter-25")
model = api.load("word2vec-google-news-300")
# cont = Contractions(kv_model=model)
# cont.load_models()

[=================================================-] 98.5% 1638.0/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
#nlp = spacy.load('en_core_web_md')
import en_core_web_sm
nlp = en_core_web_sm.load()

In [14]:
from pycontractions import Contractions

In [15]:
cont = Contractions(kv_model=model)
cont.load_models()

In [16]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [17]:
def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

In [18]:
def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())

In [19]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = list(cont.expand_texts([text], precise=True))[0]
    return text

In [20]:
def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [21]:
data['Text'] = data['Text'].apply(lambda x: " ".join(text_preprocessing(x)))
data['Label'] = (data['Label'] == 'INFORMATIVE').astype(int)

In [23]:
data

,Id,Text,Label
0,1241490299215634434,official death toll covid19 united kingdom gre...,1
1,1245916400981381130,dearest mr . president @user coronavirus death...,1
2,1241132432402849793,late update march new case new death united st...,1
3,1236107253666607104,zhen ba gong zhu bu dang gan bu breaking peo...,1
4,1239673817552879619,oklahoma city state department education annou...,0
...,...,...,...
6931,1241325232415105025,make major announcement view # coronavirus out...,0
6932,1235624084089778176,question receive ventilator require l aware sh...,1
6933,1246018213995044870,cmt air special celebrate life&music kenny rog...,0
6934,1239750367329439744,current testing procedure test pcr suspect e...,0


In [24]:
data.to_csv(data_directory + "preprocessed_train.tsv", sep='\t')

In [25]:
valid_tsv = data_directory + "valid.tsv"  #remove ‘content/’ from path then use 
valid_data = pd.read_csv(valid_tsv, sep='\t')
valid_data['Text'] = valid_data['Text'].apply(lambda x: " ".join(text_preprocessing(x)))
valid_data['Label'] = (valid_data['Label'] == 'INFORMATIVE').astype(int)
valid_data.to_csv(data_directory + "preprocessed_valid.tsv", sep='\t')

In [26]:
valid_data

,Id,Text,Label
0,1241728922192142336,say pakistan italy week confirm case pakistan ...,0
1,1235713405992030209,second case dr canadian woman identify indic...,1
2,1245941302367305728,kill chain cyber war america election document...,0
3,1245913002840391681,town host virtual towncouncil meeting technolo...,0
4,1240543259299987457,report suggest actual number undiagnosed coron...,0
...,...,...,...
995,1245955124222099456,coronavirus take hold uk earlier think data re...,0
996,1241768801210904576,talk man rowan county second confirm case covi...,1
997,1241172153040502795,governor wolf delay enforcement non - life sus...,0
998,1239740620194766848,sheriff department reduce jail population sher...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/

https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/



In [ ]:
tfidf_matrix = vectorizer.fit_transform(data['Text'].to_numpy())

In [ ]:
tfidf_matrix.shape

(6936, 16381)

In [ ]:
tfidf_matrix

<6936x16381 sparse matrix of type '<class 'numpy.float64'>'
	with 125627 stored elements in Compressed Sparse Row format>

https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/



https://medium.com/analytics-vidhya/fundamentals-of-bag-of-words-and-tf-idf-9846d301ff22


https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa

In [ ]:
#tfidf_matrix = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])

In [ ]:
from scipy import sparse

sparse.save_npz(data_directory + "tfidf_matrix.npz", tfidf_matrix)
#tfidf_matrix = sparse.load_npz("tfidf_matrix.npz")

https://towardsdatascience.com/nlp-performance-of-different-word-embeddings-on-text-classification-de648c6262b

https://www.kaggle.com/nhrade/text-classification-using-word-embeddings

In [ ]:
X = tfidf_matrix
#df['Label'] = np.where(df['Label'] == 'INFORMATIVE', 1, df['Label'])
#df['Label'] = np.where(df['Label'] == 'UNINFORMATIVE', 0, df['Label'])
data['Label'] = (data['Label'] == 'INFORMATIVE').astype(int)
y = data['Label'].to_numpy()

In [ ]:
X

<6936x16381 sparse matrix of type '<class 'numpy.float64'>'
	with 125627 stored elements in Compressed Sparse Row format>

In [ ]:
print('Y')
y.shape

Y


(6936,)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0).fit(X, y)

In [ ]:
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

0.8894175317185697

In [ ]:
#/content/drive/My Drive/MTP/data/valid.gsheet
validation_tsv = data_directory + "valid.tsv"  #remove ‘content/’ from path then use 
valid_data = pd.read_csv(validation_tsv, sep='\t')
valid_data['Text'] = valid_data['Text'].apply(lambda x: " ".join(text_preprocessing(x)))
valid_data['Label'] = (valid_data['Label'] == 'INFORMATIVE').astype(int)
#X_valid = vectorizer.fit_transform(valid_data['Text'].to_numpy())
X_valid = valid_data['Text'].to_numpy()
y_valid = valid_data['Label'].to_numpy()
clf.score(X_valid, y_valid)

ValueError: ignored

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

preprocessed_X = data['Text'].to_numpy()
txt_pipe = Pipeline([('vect', preprocessed_X), 
                     ('tfidf', TfidfTransformer()), 
                     ('mnb', MultinomialNB(fit_prior=False)),])

ValueError: ignored

In [ ]:
avg_doc = []
for val in doc_word_list:
  mean = []
  for word in val:
      mean.append(tfidf_matrix[word])
  avg_doc.append(np.array(mean).mean(axis=0))

IndexError: ignored